# 1.Trabajando con API's

La intención de este notebook es la de trabajar con API's, en este caso, para la descarga de información para su posterior analisis.

In [1]:
# Importando las librerias
import requests
import pandas as pd
import time

In [2]:
# Parametros para usar el API DE YT
# Amable recordatorio de siempre revisar esta pagina para crear la API}
# https://www.slickremix.com/docs/get-api-key-for-youtube/
# https://console.cloud.google.com/project
# XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX

API_KEY = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
CHANNEL_ID = 'UCBNvm8XDKtQhoi6usy4HXAw'

In [3]:
# Prueba de la API
response = requests.get('https://api.github.com').json()

In [4]:
# Si se imprime información vamos por buen camino
response

{'current_user_url': 'https://api.github.com/user',
 'current_user_authorizations_html_url': 'https://github.com/settings/connections/applications{/client_id}',
 'authorizations_url': 'https://api.github.com/authorizations',
 'code_search_url': 'https://api.github.com/search/code?q={query}{&page,per_page,sort,order}',
 'commit_search_url': 'https://api.github.com/search/commits?q={query}{&page,per_page,sort,order}',
 'emails_url': 'https://api.github.com/user/emails',
 'emojis_url': 'https://api.github.com/emojis',
 'events_url': 'https://api.github.com/events',
 'feeds_url': 'https://api.github.com/feeds',
 'followers_url': 'https://api.github.com/user/followers',
 'following_url': 'https://api.github.com/user/following{/target}',
 'gists_url': 'https://api.github.com/gists{/gist_id}',
 'hub_url': 'https://api.github.com/hub',
 'issue_search_url': 'https://api.github.com/search/issues?q={query}{&page,per_page,sort,order}',
 'issues_url': 'https://api.github.com/issues',
 'keys_url': '

## 1.1 Llamada a la API

La estructura de la URL para consultar la data de YT es la siguiente

'https://www.googleapis.com/youtube/v3/search?key='+**API_KEY**+"&channelId="+**CHANNEL_ID**+"&part=snippet,id&order=date&maxResults=10000"

* API_KEY es la key que se genero anteriormente

* CHANNEL_ID del canal objetivo

Para más información, consultar la documentación oficial: https://developers.google.com/youtube/v3/docs/search

In [72]:
# en este caso me interesa buscar la información del canal de Dross
# el CHANNEL_ID es el siguiente UCNYW2vfGrUE6R5mIJYzkRyQ
# en la parte de Results=10000 estamos pidiendo la informacion de los 10000 videos subidos (en donde espero que no exista gente loca que suba tantos videos)

channel_objetive = 'UCNYW2vfGrUE6R5mIJYzkRyQ'
pageToken = ''

url = 'https://www.googleapis.com/youtube/v3/search?key='+API_KEY+'&channelId='+channel_objetive+'&part=snippet,id&order=date&maxResults=10000'+pageToken

In [6]:
# Verificamos si hay respuesta con los datos necesarios
response = requests.get(url=url).json()

In [7]:
response.keys()

dict_keys(['kind', 'etag', 'nextPageToken', 'regionCode', 'pageInfo', 'items'])

In [13]:
# Se observa el formato JSON del ultimo video subido por Dross

response['items'][0]

{'kind': 'youtube#searchResult',
 'etag': 'kMg6_f0L5Nuy1FYtoQdSJvdrC-s',
 'id': {'kind': 'youtube#video', 'videoId': '-USeILTeqDU'},
 'snippet': {'publishedAt': '2023-03-01T01:00:03Z',
  'channelId': 'UCNYW2vfGrUE6R5mIJYzkRyQ',
  'title': 'Hay algo extraño EN CASA DE SEAN',
  'description': 'http://bit.ly/1a1sm3k #dross.',
  'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/-USeILTeqDU/default.jpg',
    'width': 120,
    'height': 90},
   'medium': {'url': 'https://i.ytimg.com/vi/-USeILTeqDU/mqdefault.jpg',
    'width': 320,
    'height': 180},
   'high': {'url': 'https://i.ytimg.com/vi/-USeILTeqDU/hqdefault.jpg',
    'width': 480,
    'height': 360}},
  'channelTitle': 'DrossRotzank',
  'liveBroadcastContent': 'none',
  'publishTime': '2023-03-01T01:00:03Z'}}

## 1.2 Seleccionando informacion

Apartir del JSON generado, es posible seleccionar determinada información, deacuerdo a las necesidades que se requieran

In [14]:
# Obtenemos el Id, titulo y fecha de subida del ultimo video publicado por Dross

video_id = response['items'][0]['id']['videoId']

video_title = response['items'][0]['snippet']['title']
video_title = str(video_title).replace('&amp;', '')

upload_date = response['items'][0]['snippet']['publishedAt']
upload_date = str(upload_date).split('T')[0]

print(f'''ID del video: {video_id}
Titulo del video: {video_title}
Fecha de subida: {upload_date}''')


ID del video: -USeILTeqDU
Titulo del video: Hay algo extraño EN CASA DE SEAN
Fecha de subida: 2023-03-01


## 1.3 Recolección de información

In [17]:
dataframe = pd.DataFrame(columns=['video_id', 'video_title', 'upload_date',
                                  'view_count', 'like_count', 'comment_count'])

In [18]:
dataframe

,video_id,video_title,upload_date,view_count,like_count,comment_count


In [24]:
for info_video in response['items']:
    # Verificamos que todos los resultados sean videos
    if info_video['id']['kind'] == 'youtube#video':
        video_id = info_video['id']['videoId']

        video_title = info_video['snippet']['title']
        video_title = str(video_title).replace('&amp;', '')

        upload_date = info_video['snippet']['publishedAt']
        upload_date = str(upload_date).split('T')[0]
        
        url_video_stats = "https://www.googleapis.com/youtube/v3/videos?id="+video_id+"&part=statistics&key="+API_KEY
        response_video_stats = requests.get(url_video_stats).json()
        
        view_count = response_video_stats['items'][0]['statistics']['viewCount']
        like_count = response_video_stats['items'][0]['statistics']['likeCount']
        comment_count = response_video_stats['items'][0]['statistics']['commentCount']
        
        column = pd.DataFrame([[video_id, video_title, upload_date,
                               view_count, like_count, comment_count]],
                              columns=['video_id', 'video_title', 'upload_date',
                                       'view_count', 'like_count', 'comment_count'])
        
        dataframe = pd.concat([dataframe, column])
        

print(f'''ID del video: {video_id}
Titulo del video: {video_title}
Fecha de subida: {upload_date}
Vistas: {view_count}
Likes: {like_count}
Comentarios: {comment_count}\n''')

dataframe.head(10)


ID del video: jlExbCOWEDc
Titulo del video: 3 videos que harán que no quieras apagar la luz de noche
Fecha de subida: 2022-10-03
Vistas: 1950356
Likes: 191439
Comentarios: 9894



,video_id,video_title,upload_date,view_count,like_count,comment_count
0,-USeILTeqDU,Hay algo extraño EN CASA DE SEAN,2023-03-01,182488,27628,2081
0,lrLZUkSL79A,La destrucción mediática de Michael Jackson,2023-02-26,1197212,107466,4733
0,9tsrQsOGV4M,7 Personas Históricas Que Vendieron Su Alma Al...,2023-02-24,151079,8149,163
0,cFlk_Rd1DGg,TOP: Las 7 personas de Twitch más...,2023-02-22,1726384,162552,10606
0,NjR_bIwIU5s,7 hechos terroríficos que no tienen explicació...,2023-02-20,156956,9052,153
0,cR1duorFOAU,3 videos para ver con la luz apagada,2023-02-19,1677670,117087,5766
0,U_phwCcoqCI,Llamando a números de los RED ROOMS,2023-02-16,1307039,92036,3423
0,V6wNi4KIMZ0,Los 7 secretos más oscuros del Vaticano Dross ...,2023-02-14,178678,9844,174
0,Oo-LwX7nGPw,TOP: 7 lugares malditos de Google Maps,2023-02-13,2426940,153627,6001
0,Eo2_kuYxpIg,3 Videos No Aptos Para Sensibles,2023-02-10,1792032,136026,5151


In [60]:
video_id = '-USeILTeqDU'

In [45]:
url = "https://youtube.googleapis.com/youtube/v3/videos?part=snippet&id="+video_id+'&key='+API_KEY
response = requests.get(url=url).json()
response

{'kind': 'youtube#videoListResponse',
 'etag': 'UZ7C-c8x8eaA_jKkCDtVz4jbJjE',
 'items': [{'kind': 'youtube#video',
   'etag': 'G3TGpFW45DH9pD8yQp_RbY4UDFQ',
   'id': '-USeILTeqDU',
   'snippet': {'publishedAt': '2023-03-01T01:00:03Z',
    'channelId': 'UCNYW2vfGrUE6R5mIJYzkRyQ',
    'title': 'Hay algo extraño EN CASA DE SEAN',
    'description': 'http://bit.ly/1a1sm3k \n#dross',
    'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/-USeILTeqDU/default.jpg',
      'width': 120,
      'height': 90},
     'medium': {'url': 'https://i.ytimg.com/vi/-USeILTeqDU/mqdefault.jpg',
      'width': 320,
      'height': 180},
     'high': {'url': 'https://i.ytimg.com/vi/-USeILTeqDU/hqdefault.jpg',
      'width': 480,
      'height': 360},
     'standard': {'url': 'https://i.ytimg.com/vi/-USeILTeqDU/sddefault.jpg',
      'width': 640,
      'height': 480},
     'maxres': {'url': 'https://i.ytimg.com/vi/-USeILTeqDU/maxresdefault.jpg',
      'width': 1280,
      'height': 720}},
    'channelTit

In [71]:
url = "https://youtube.googleapis.com/youtube/v3/videos?part=snippet&id="+video_id+'&key='+API_KEY
response = requests.get(url=url).json()

video_title = response['items'][0]['snippet']['title']
video_title = str(video_title).replace('&amp;', '')

upload_date = response['items'][0]['snippet']['publishedAt']
upload_date = str(upload_date).split('T')[0]

url = "https://www.googleapis.com/youtube/v3/videos?id="+video_id+"&part=statistics&key="+API_KEY
response = requests.get(url=url).json()

view_count = response['items'][0]['statistics']['viewCount']
like_count = response['items'][0]['statistics']['likeCount']
comment_count = response['items'][0]['statistics']['commentCount']

print(video_id)
print(video_title)
print(upload_date)
print(view_count)
print(like_count)
print(comment_count)

#print(response['items'][0]['snippet']['title'])
#print(response['items'][0]['snippet']['publishedAt'])

-USeILTeqDU
Hay algo extraño EN CASA DE SEAN
2023-03-01
271747
35193
2463


In [ ]:
# In progress is up

def get_video_details(video_id):
    url = "https://www.googleapis.com/youtube/v3/videos?id="+video_id+"&part=statistics&key="+API_KEY
    
    response = requests.get(url=url).json()
    
    column = pd.DataFrame([[video_id, video_title, upload_date,
                               view_count, like_count, comment_count]],
                              columns=[video_id, 'video_title', 'upload_date',
                                       'view_count', 'like_count', 'comment_count'])
    
    return pd.DataFrame([[response['items'][0]['st']]],
                        columns=['video_id', 'video_title', 'upload_date',
                                       'view_count', 'like_count', 'comment_count'])